In [ ]:
import os, sys
import scanpy as sc
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import matplotlib.cm as cm
from STutils.pl import getDefaultColors

matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42
matplotlib.rcParams["font.serif"] = ["Arial"]
sc.settings.set_figure_params(
    dpi=100, dpi_save=300, frameon=False, facecolor="white", fontsize=16, vector_friendly=True, figsize=(5, 5)
)
sc._settings.ScanpyConfig(figdir="./", n_jobs=30)

## Sample summary 

In [ ]:
# 空间组样本热图
os.chdir("/jdfsbjcas1/ST_BJ/P21H28400N0232/gongchanghao/project/PDAC/spatial/cellbin_v3/1.1_sample_summary")
df = pd.read_csv("sample_info.xls", sep="\t")
df.set_index("sampleName", inplace=True)
features = ["SampleType", "scRNA-seq", "Stereo-seq", "Gender", "Stage"]
df = df[features]
classtonum = {"Tum": 3, "PT": 2, "yes": 4, "no": 5, "female": 0, "male": 1, "IB": 6, "IIB": 7, "III": 8}
df = pd.DataFrame(
    np.array([classtonum[i] for i in df.values.flat]).reshape(df.shape), index=df.index, columns=df.columns
)
df

In [ ]:
# 颜色设置
colors1 = [colorlist1[i] for i in [1, 0]]
colors2 = ["#9af764", "lightgrey"]
colors3 = ["#3c9992", "lightgrey"]
colors4 = [colorlist1[i] for i in [3, 4]]
colors5 = [colorlist2[i] for i in [15, 13, 12]]
from matplotlib.colors import ListedColormap

# cmap1 = ListedColormap(colors1)
cmap1 = ListedColormap(colors1)
cmap2 = ListedColormap(colors2)
cmap3 = ListedColormap(colors3)
cmap4 = ListedColormap(colors4)
cmap5 = ListedColormap(colors5)
cmaplist = [cmap1, cmap2, cmap3, cmap4, cmap5]

In [ ]:
from matplotlib.patches import Patch

# 绘图
fig, ax = plt.subplots(figsize=(10, 4))
ax.grid(False)
for i, col in enumerate(df.index):
    for j, index in enumerate(df.columns):
        ax.add_patch(plt.Rectangle((i - 0.5, j - 0.5), 1, 1, fill=None, alpha=1, edgecolor="gray"))
ax.matshow(
    df.mask(((df == df) | df.isnull()) & (df.columns != "SampleType")).T, cmap=cmap1, aspect=1
)  # You can change the colormap here
ax.matshow(df.mask(((df == df) | df.isnull()) & (df.columns != "scRNA-seq")).T, cmap=cmap2, aspect=1)
ax.matshow(df.mask(((df == df) | df.isnull()) & (df.columns != "Stereo-seq")).T, cmap=cmap3, aspect=1)
ax.matshow(df.mask(((df == df) | df.isnull()) & (df.columns != "Gender")).T, cmap=cmap4, aspect=1)
ax.matshow(df.mask(((df == df) | df.isnull()) & (df.columns != "Stage")).T, cmap=cmap5, aspect=1)
legend_elements1 = [
    Patch(facecolor=colors1[1], edgecolor=colors1[1], label="Tum"),
    Patch(facecolor=colors1[0], edgecolor=colors1[0], label="PT"),
]
legend_elements2 = [
    Patch(facecolor=colors2[0], edgecolor=colors2[0], label="scRNA_seq"),
    Patch(facecolor=colors3[0], edgecolor=colors3[0], label="Stereo_seq"),
    Patch(facecolor="lightgrey", edgecolor="lightgrey", label="None"),
]
legend_elements3 = [
    Patch(facecolor=colors4[1], edgecolor=colors4[1], label="female"),
    Patch(facecolor=colors4[0], edgecolor=colors4[0], label="male"),
]
legend_elements4 = [
    Patch(facecolor=colors5[0], edgecolor=colors5[0], label="IB"),
    Patch(facecolor=colors5[1], edgecolor=colors5[1], label="IIB"),
    Patch(facecolor=colors5[2], edgecolor=colors5[2], label="III"),
]
# for i, col in enumerate(df.columns):
#     legend_elements.append(plt.Line2D([0], [0], marker='o', color='w', label=col, markerfacecolor=cmaplist[i](1)))
ax.tick_params(axis="both", which="both", bottom=False, left=False)
plt.xticks(range(df.index.shape[0]), df.index, rotation=45, fontsize=10)
plt.yticks(range(df.columns.shape[0]), df.columns)
legend1 = plt.legend(handles=legend_elements1, loc="upper left", bbox_to_anchor=(0, -0.1), title="SampleType")
legend2 = plt.legend(handles=legend_elements2, loc="upper center", bbox_to_anchor=(0.35, -0.1), title="Sequencing")
legend3 = plt.legend(handles=legend_elements3, loc="upper right", bbox_to_anchor=(0.65, -0.1), title="Gender")
legend4 = plt.legend(handles=legend_elements4, loc="upper right", bbox_to_anchor=(0.85, -0.1), title="Stage")
ax.add_artist(legend1)
ax.add_artist(legend2)
ax.add_artist(legend3)
ax.add_artist(legend4)
plt.savefig("sample_summary.pdf", bbox_inches="tight", bbox_extra_artists=(legend1, legend2, legend3))

## sc-RNA seq overview

In [ ]:
# 样本编号信息
import yaml

with open("/jdfsbjcas1/ST_BJ/P21H28400N0232/gongchanghao/project/PDAC/spatial/cellbin_v3/sample.yaml", "r") as file:
    sample_dict = yaml.safe_load(file)

In [ ]:
adata = sc.read(
    "/jdfsbjcas1/ST_BJ/P21H28400N0232/gongchanghao/project/PDAC/spatial/cellbin_v3/1.1_cellbin_rawdata/sc_obj4.h5ad"
)

In [ ]:
color_palette_fl = open(
    "/jdfsbjcas1/ST_BJ/P21H28400N0232/gongchanghao/project/PDAC/spatial/cellbin_v3/1.6_sc_summary/color_celltype.list",
    "r",
).readlines()
color_palette = []
for line in color_palette_fl:
    color_palette.append(line.strip().split("\t")[1])
sc.settings.set_figure_params(
    dpi=100, dpi_save=300, frameon=False, facecolor="white", fontsize=13, vector_friendly=True, figsize=(5, 5)
)
sc.pl.umap(
    adata,
    color=["CellType"],
    wspace=0.4,
    palette=color_palette,
    show=False,
    save="scRNA_umap_celltype.pdf",
    legend_loc="on data",
    legend_fontsize=12,
)
sc.settings.set_figure_params(
    dpi=100, dpi_save=300, frameon=False, facecolor="white", fontsize=13, vector_friendly=True, figsize=(6, 5)
)
sc.pl.umap(adata, color=["sample"], wspace=0.4, palette="tab20", show=False, save="scRNA_umap_sample.pdf")

In [ ]:
from adjustText import adjust_text


def gen_mpl_labels(adata, groupby, exclude=(), ax=None, adjust_kwargs=None, text_kwargs=None):
    if adjust_kwargs is None:
        adjust_kwargs = {"text_from_points": False}
    if text_kwargs is None:
        text_kwargs = {}

    medians = {}

    for g, g_idx in adata.obs.groupby(groupby).groups.items():
        if g in exclude:
            continue
        medians[g] = np.median(adata[g_idx].obsm["X_umap"], axis=0)

    if ax is None:
        texts = [plt.text(x=x, y=y, s=k, **text_kwargs) for k, (x, y) in medians.items()]
    else:
        texts = [ax.text(x=x, y=y, s=k, **text_kwargs) for k, (x, y) in medians.items()]

    adjust_text(texts, **adjust_kwargs)


with plt.rc_context({"figure.figsize": (4, 4), "figure.dpi": 300, "figure.frameon": False}):
    ax = sc.pl.umap(adata, color="CellType", title="CellTypes", show=False, legend_loc=None, frameon=False)
    gen_mpl_labels(
        adata,
        "CellType",
        exclude=("None",),  # This was before we had the `nan` behaviour
        ax=ax,
        # adjust_kwargs=dict(arrowprops=dict(arrowstyle='-', color='black')),
        text_kwargs=dict(fontsize=7),
    )
    fig = ax.get_figure()
    fig.tight_layout()
    plt.savefig("scRNA_umap_celltype.pdf", bbox_inches="tight")

In [ ]:
matplotlib.rcParams["font.size"] = 12
matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42
matplotlib.rcParams["font.serif"] = ["Arial"]
matplotlib.rcParams["legend.fontsize"] = 12
filtered_data = adata.obs
grouped = filtered_data.groupby(["sample", "CellType"]).size().unstack("CellType").fillna(0)
proportions = grouped.divide(grouped.sum(axis=1), axis=0)
# sort proportions by sample
proportions = proportions.sort_index(axis=0)
fig, axs = plt.subplots(figsize=(8, 5))
num_colors = len(proportions.columns)
color_palette_fl = open(
    "/jdfsbjcas1/ST_BJ/P21H28400N0232/gongchanghao/project/PDAC/spatial/cellbin_v3/1.6_sc_summary/color_celltype.list",
    "r",
).readlines()
color_palette = []
for line in color_palette_fl:
    color_palette.append(line.strip().split("\t")[1])
proportions.plot(kind="bar", stacked=True, ax=axs, width=0.8, color=color_palette)
axs.set_title("Cell Type Proportions by Sample")
axs.set_xlabel("Samples")
axs.set_ylabel("Proportions")
axs.tick_params(axis="x", labelsize=12)
axs.legend(title="Cell Type", bbox_to_anchor=(1.05, 1), loc="upper left", frameon=False)
axs.grid(False)
plt.setp(axs.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
fig.savefig(f"scRNA_percent_celltype_sample.pdf", bbox_inches="tight")

In [ ]:
sc.tl.rank_genes_groups(adata,
                        'CellType',
                        method='wilcoxon',
                        use_raw=False,
                        # layer='norm',
                        key_added='rank_genes_groups')

from collections import OrderedDict

DEG_dict = OrderedDict(
    [
        ("Malignant", ["S100P", "FXYD3", "S100A6", "KRT19", "EPCAM"]),
        ("Epi", ["CELA3A", "CPB1", "PIGR", "FXYD2", "CFTR"]),
        ("Myeloid", ["CD163", "CSF1R", "C1QB", "AIF1", "TYROBP"]),
        ("Lymphoid", ["CD3D", "GZMA", "GNLY", "MS4A1", "JCHAIN"]),
        ("Mysen", ["COL1A2", "DCN", "VWF", "CLEC14A", "RGS5"]),
    ]
)
sc.pl.dotplot(adata, DEG_dict, groupby="CellType", cmap="viridis_r", save="scRNA_DEG_dotplot.pdf", figsize=(20, 4))

# spatial overview
## 统计cellbin基因表达平均数、中位数和UMI平均数、中位数

In [ ]:
from spatial.plot_cellbin import plot_cellbin_gradient
for sample in samplelist:
    adata = sc.read(
        f"/jdfsbjcas1/ST_BJ/P21H28400N0232/gongchanghao/project/PDAC/spatial/rawdata/{sample}/{sample}.h5ad"
    )
    adata.obs_names._data = np.char.replace(adata.obs_names._data.astype(str), ".0", "").astype(object)
    mask = f"/jdfsbjcas1/ST_BJ/P21H28400N0232/gongchanghao/project/PDAC/spatial/rawdata/{sample}/{sample}_regist_mask_ft.tif"
    adata.var_names_make_unique()
    adata.var["mt"] = adata.var_names.str.startswith("MT-")
    sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True)
    plot_cellbin_gradient(adata, mask, "n_genes_by_counts", sample_dict[sample])
    plot_cellbin_gradient(adata, mask, "total_counts", sample_dict[sample])

## 空间降维可视化

In [ ]:
adata = sc.read(
    "/jdfsbjcas1/ST_BJ/P21H28400N0232/gongchanghao/project/PDAC/spatial/cellbin_v3/6.9_Tumor_sc_scvi_cluster/adata_big_cell_scvi.h5ad"
)

In [ ]:
SCVI_LATENT_KEY = "X_scVI"
sc.pp.neighbors(adata, use_rep=SCVI_LATENT_KEY)
sc.tl.leiden(adata, key_added="scvi_leiden", resolution=0.6)
sc.tl.umap(adata)
sc.pl.umap(adata, color=["cellTypes_nmf", "sample", "scvi_leiden"], save="_region_scvi.pdf", ncols=1)

## 空间细胞类型比例

In [ ]:
adata = sc.read(
    "/jdfsbjcas1/ST_BJ/P21H28400N0232/gongchanghao/project/PDAC/spatial/cellbin_v3/1.1_cellbin_rawdata/filtered_data/merged_adata4.h5ad"
)
matplotlib.rcParams["font.size"] = 12
matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42
matplotlib.rcParams["font.serif"] = ["Arial"]
matplotlib.rcParams["legend.fontsize"] = 12
# 画celltype比例图
filtered_data = adata.obs
grouped = filtered_data.groupby(["sample", "celltype"]).size().unstack("celltype").fillna(0)
proportions = grouped.divide(grouped.sum(axis=1), axis=0)
fig, axs = plt.subplots(figsize=(6, 5))
num_colors = len(proportions.columns)
# color_palette = getDefaultColors(num_colors, type=1)
color_palette_fl = open(
    "/jdfsbjcas1/ST_BJ/P21H28400N0232/gongchanghao/project/PDAC/spatial/cellbin_v3/1.2_cellbin_spatial_plot/celltype_spatial_plot/color_celltype.list",
    "r",
).readlines()
color_palette = []
for line in color_palette_fl:
    color_palette.append(line.strip().split("\t")[1])
proportions.plot(kind="bar", stacked=True, ax=axs, color=color_palette, width=0.8)
axs.set_title("Cell Type Proportions by Sample")
axs.set_xlabel("Samples")
axs.set_ylabel("Proportions")
axs.tick_params(axis="x", labelsize=12)
axs.legend(title="Cell Type", bbox_to_anchor=(1.05, 1), loc="upper left", frameon=False)
axs.grid(False)
plt.setp(axs.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
fig.savefig(f"percent_celltype_sample.pdf", bbox_inches="tight")

## metacell  celltype DEGs

In [ ]:
adata = sc.read_h5ad(
    "/jdfsbjcas1/ST_BJ/P21H28400N0232/gongchanghao/project/PDAC/spatial/cellbin_v3/2.1_merge_big_cell/all_merged_adata.h5adbak"
)

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata = removeBiasGenes(adata)
sc.tl.rank_genes_groups(adata,
                        'celltype',
                        method='wilcoxon',
                        use_raw=False,
                        # layer='norm',
                        key_added='rank_genes_groups')
DEG_dict = getDEG(
    adata,
    "celltype",
    qval_cutoff=0.05,
    mean_expr_cutoff=1,
    #   layer='norm',
    top_genes=200,
    save=f"DEG_volcano_celltype_all.png",
)

In [ ]:
degfl = open(f"DEG_celltype.xls", "w")
for key, value in DEG_dict.items():
    degfl.write(key + "\t" + "\t".join(value) + "\n")
degfl.close()
result = adata.uns["rank_genes_groups"]
groups = result["names"].dtype.names
df = pd.DataFrame(
    {
        group + "_" + key: result[key][group]
        for group in groups
        for key in ["names", "logfoldchanges", "pvals", "pvals_adj"]
    }
)
df.to_csv(f"DEG_wolcoxon_celltype.csv")

In [ ]:
color_palette_fl = open(
    "/jdfsbjcas1/ST_BJ/P21H28400N0232/gongchanghao/project/PDAC/spatial/cellbin_v3/1.2_cellbin_spatial_plot/celltype_spatial_plot/color_celltype.list",
    "r",
).readlines()
color_palette = []
for line in color_palette_fl:
    color_palette.append(line.strip().split("\t")[1])
sc.settings.set_figure_params(
    dpi=100, dpi_save=300, facecolor="white", frameon=False, fontsize=12, vector_friendly=True, figsize=(3, 3)
)
sc.pl.rank_genes_groups_stacked_violin(
    adata,
    n_genes=5,
    #    cmap='viridis_r',
    row_palette=color_palette,
    save="celltype_deg.pdf",
    swap_axis=True,
    dendrogram=False,
)

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    adata,
    n_genes=5,
    dendrogram=False,
    color_map="viridis_r",
    swap_axes=False,
    standard_scale="var",
    save="DEG_celltype.pdf",
)